# Introduction
What's the last school shooting you remember? If you're on top of your news, then you'll know it was the Pleasantville, New Jersey shooting, a few weeks ago (from the time this was written). 
Do you remember the details, though?
Probably not.

What about the shooting the day right before Pleasantville? Or the two in October?
The problem is clear: school shootings are a regular occurrence and it's hard to keep track of them all, let alone the details of each one.
In fact, according to Wikipedia, there were 64 US school shootings from 2000-2009, 87 from 2010-2014, and 113 from 2015-November 2019. 
If these numbers seem lower than you expected, remember that these are ONLY school shootings, not mass shootings. The numbers there are staggering: 2,138 since 2013, roughly one a day. The number changes a bit depending on how you define a mass shooting, but either way the point remains: the number of shootings that occur in the U.S. is unacceptably high.

For this project we want to focus on school shootings, as the definition of school shooting is much more clearly defined and can be analyzed easier with a smaller timeframe. Our goal is to find causes of the spike in shootings, especially in the past decade.
It's important to note that none of us want to push any sort of policy or idea or agenda here: we want to stay as neutral as possible.


# References
https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States
https://en.wikipedia.org/wiki/Mass_shootings_in_the_United_States
https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States_(before_2000)#20th_century
https://en.wikipedia.org/wiki/List_of_unsuccessful_attacks_related_to_schools


## Beginning our Analysis
Our goal is to figure out significant school shootings by getting compiled data from Wikipedia and recording certain characteristics about that specific event. With those characteristics, we want to scrape various data sources, such as Fox News, CNN, etc, and see if characteristics of the shooting affect the amount that it is reported among various source.
We recognize that because we have a very small dataset that we may not be able to come up with any significant conclusions, but we believe that it is a start to making steps towards doing so in identifying these reporting patterns.

In [18]:
import pandas as pd
import requests as rq
import numpy as np
import re
from datetime import datetime
from bs4 import BeautifulSoup


In [19]:
def makeDF(header, body):
    sol = pd.DataFrame.from_records(body)
    sol.columns = header
    return sol

def parseWiki(url):
    school_shooting_url = rq.get(url).text
    soup = BeautifulSoup(school_shooting_url)
    soup.prettify()
    # sortable wikitable is what wikipedia has their tables called
    tables = soup.find_all("table",{"class":"sortable wikitable"})
    header = []
    body = []
    # wikipedia separates by year chunks so we iterate through
    for table in tables:
        for row in table.find_all("tr"):
            temp = []
            if not header:
                for h in row.find_all("th"):
                    header.append(h.get_text().rstrip())
            for col in row.find_all("td"):
                cur = col.get_text()
                # want to get rid of wiki references
                cur = re.sub(r'\[.*\]', '', cur)
                cur = cur.replace("\n", " ")
                noline = cur.rstrip()
                temp.append(noline)
            body.append(temp)
    # the first is the header, so we take it out
    return header, body[1:]


In [20]:
header, body = parseWiki("https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States")
df = makeDF(header, body)
df.head()

,Date,Location,Deaths,Injuries,Description
0,"February 29, 2000","Flint, Michigan",1,0,Shooting of Kayla Rolland: At Buell Elementary...
1,"May 26, 2000","Lake Worth, Florida",1,0,"13-year-old honor student, Nathaniel Brazill, ..."
2,"June 28, 2000","Seattle, Washington",2,0,58-year-old Director of the Division of Pathol...
3,"August 28, 2000","Fayetteville, Arkansas",2,0,"36-year-old James Easton Kelly, a PhD candidat..."
4,"September 26, 2000","New Orleans, Louisiana",0,2,"13 year-olds Darrel Johnson, and Alfred Anders..."


In [21]:
df.tail()

,Date,Location,Deaths,Injuries,Description
266,"October 22, 2019","Santa Rosa, California",0,1,"A 16-year-old student was shot twice, at least..."
267,"November 14, 2019","Santa Clarita, California",3,3,Saugus High School shooting: Five people were ...
268,"November 15, 2019","Pleasantville, New Jersey",1,2,Five men opened fire during a playoff game bet...
269,"November 23, 2019","Union City, California",2,0,"Two boys, aged 11 and 14-years-old were shot i..."
270,"November 27, 2019","Vancouver, Washington",2,1,Tiffany Hill who was fatally shot by her estra...


Here we look at unnsuccessful attacks in schools:

In [22]:
def parseWiki(url):
    school_shooting_url = rq.get(url).text
    soup = BeautifulSoup(school_shooting_url)
    soup.prettify()
    
    # It's called a wikitable sortable here, for whatever reason.
    tables = soup.find_all("table",{"class":"wikitable sortable"})
    header = []
    body = []
    # wikipedia separates by year chunks so we iterate through
    for table in tables:
        
        for row in table.find_all("tr"):
            temp = []
            if not header:
                for h in row.find_all("th"):
                    header.append(h.get_text().rstrip())
            for col in row.find_all("td"):
                cur = col.get_text()
                # want to get rid of wiki references
                cur = re.sub(r'\[.*\]', '', cur)
                cur = cur.replace("\n", " ")
                noline = cur.rstrip()
                temp.append(noline)
            body.append(temp)
        
    # the first is the header, so we take it out
    return header, body[1:]


header, body = parseWiki("https://en.wikipedia.org/wiki/List_of_unsuccessful_attacks_related_to_schools")
dfsuccess = makeDF(header, body)
dfsuccess.head()

,Date,Location,Description
0,"October 12, 1992","Lincoln, Nebraska, United States","A 43-year-old graduate student, Arthur McElroy..."
1,"November 16, 1998","Burlington, Wisconsin, United States",Five students were arrested the morning they w...
2,May 1999,"Port Huron, Michigan, United States","A 12-year-old, 13-year-old and two 14-year-old..."
3,"January 30, 2001","Cupertino, California, United States",De Anza College student Al DeGuzman planned a ...
4,"February 14, 2001","Elmira, New York, United States","Jeremy Getman, 18, planned a school attack at ..."


In [23]:
dfsuccess.tail()

,Date,Location,Description
22,"May 1, 2014","Waseca, Minnesota, United States",17-year-old John David LaDue
23,"November 3, 2014","Newcastle upon Tyne, United Kingdom",18-year-old Liam Lyburd was arrested at his ho...
24,"December 1, 2014","Plain City, Utah, United States",A 16-year-old at Fremont High School was arres...
25,"February 14, 2018","Everett, Washington, United States","An 18-year-old, Joshua O'Connor, was arrested ..."
26,"April 17, 2019","Jefferson County, Colorado, United States","An 18-year-old, Sol Pais, flew to Colorado fro..."


### Using APIs to get news sources and attempt to parse data
Now that we have the wikipedia data parsed, we want to set up the data scraping from various sources. We will be first using google news and webscraping articles that are at a certain date and do some more analysis. 

In [24]:
def parseResp(**params):
    url = "https://www.google.com/search?q={query}&tbs=cdr%3A1%2Ccd_min%3A{month}%2F{from_day}%2F{year}%2Ccd_max%3A{month}%2F{to_day}%2F{year}"
    url = "https://www.google.com/search?pz=1&cf=all&ned=us&hl=en&tbm=nws&gl=us&as_q={query}&as_occt=any&as_drrb=b&as_mindate={month}%2F%{from_day}%2F{year}&as_maxdate={month}%2F{to_day}%2F{year}&tbs=cdr%3A1%2Ccd_min%3A3%2F1%2F13%2Ccd_max%3A3%2F2%2F13&authuser=0"
    response = rq.get(url.format(**params))
    return response

# df: the dataframe that has date, location, injuries, deaths, description of shooting
def googlenews(df):
    for i, row in df.iterrows():
        date = datetime.strptime(row['Date'], '%B %d, %Y')
        resp = parseResp(query = "test", month = date.month, from_day = date.day, to_day = date.day + 1, year = date.year)
        print(resp.json())

googlenews(makeDF(header, body))


JSONDecodeError: Expecting value: line 1 column 1 (char 0)